In [1]:
import pandas as pd
import numpy as np
import cv2
import os

import time

### params

In [ ]:
directory = 'clean image/full_selection_sel' # select the folder

## prototype

### delete grayscale images

In [2]:
directory = 'clean image/full_selection_sel' # select the folder
counter = 0

for filename in os.listdir(directory):
    r = directory + '/' + filename
    image = cv2.imread(r)
    (B, G, R) = cv2.split(image)

    if (B==G).all() == True:
        os.remove(r)
        counter += 1

print(f'{counter} grayscale images deleted.')

310 grayscale images deleted.


### dimension of images

In [3]:
# create dataframe of dimension of the images

# initialte a dataframe for dimension
keys = ['Object ID', 'width','height']
df_dim = pd.DataFrame(columns=keys)

# params
directory = 'clean image/full_selection_sel' # select the folder

# function
for filename in os.listdir(directory):
    r = directory + '/' + filename
    image = cv2.imread(r)
    new_row = {'Object ID':filename, 'width':image.shape[1],'height':image.shape[0]}
    new_row = pd.DataFrame(new_row,index=[0])
    df_dim = pd.concat([df_dim, new_row], ignore_index=True)

In [4]:
# clean the .jpg in df
df_dim['Object ID'] = df_dim['Object ID'].map(lambda x: x[:-4])

In [10]:
# create a list of our conditions
conditions = [
    df_dim['height'] / df_dim['width'] >=1.1,
    df_dim['height'] / df_dim['width'] <=0.9,
    (df_dim['height'] / df_dim['width'] >0.9) & (df_dim['height'] / df_dim['width'] <1.1) 
    ]

# create a list of the values we want to assign for each condition
values = ['vertical', 'horizontal', 'square']

# create a new column and use np.select to assign values to it using our lists as arguments
df_dim['ratio'] = np.select(conditions, values)

# display updated DataFrame
df_dim.head()

,Object ID,width,height,ratio
0,100017,991,1795,vertical
1,100019,1800,1898,square
2,100020,1440,2526,vertical
3,100021,1663,1960,vertical
4,100024,1225,2838,vertical


In [11]:
ra = df_dim['ratio'].value_counts()

In [22]:
ra.to_dict()

{'vertical': 2081, 'horizontal': 295, 'square': 189}

## define functions

### delete grayscale images in folder

In [ ]:
# directory = 'clean image/full_selection_sel' # select the folder

def delete_gray_image_in_folder(directory):
    start = time.time() # start timer
    counter = 0 # handy counter

    for filename in os.listdir(directory):
        r = directory + '/' + filename
        image = cv2.imread(r)
        (B, G, R) = cv2.split(image)

        if (B==G).all() == True: # RBG channels are same for grayscales
            os.remove(r)
            counter += 1

    print(f'{counter} grayscale images deleted.')
    print(f'{time.time()-start_time} seconds slipped...') # print out the time spent

### create a df for dimension checkout

In [28]:
# directory = 'clean image/full_selection_sel' # select the folder

def dimension_df(directory):
    """return a dataframe"""

    # initialte a dataframe for dimension
    keys = ['Object ID', 'width','height']
    df_dim = pd.DataFrame(columns=keys)

    # function
    for filename in os.listdir(directory):
        r = directory + '/' + filename
        image = cv2.imread(r)
        new_row = {'Object ID':filename, 'width':image.shape[1],'height':image.shape[0]}
        new_row = pd.DataFrame(new_row,index=[0])
        df_dim = pd.concat([df_dim, new_row], ignore_index=True)

    # clean the .jpg in df
    df_dim['Object ID'] = df_dim['Object ID'].map(lambda x: x[:-4])

    # create a list of our conditions
    conditions = [
        df_dim['height'] > df_dim['width'],
        df_dim['height'] < df_dim['width'],
        df_dim['height'] == df_dim['width']
        ]

    # conditions for rough square
    # a rough square doesn't need to be cropped using other tools
    # if there are many rough square, it will save some processing time
    conditions2 = [
        df_dim['height'] / df_dim['width'] >=1.1,
        df_dim['height'] / df_dim['width'] <=0.9,
        (df_dim['height'] / df_dim['width'] >0.9) & (df_dim['height'] / df_dim['width'] <1.1) 
        ]

    # create a list of the values we want to assign for each condition
    values = ['vertical', 'horizontal', 'square']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_dim['ratio'] = np.select(conditions, values)
    df_dim['rough ratio'] = np.select(conditions2, values)


    ra = df_dim['ratio'].value_counts().to_dict()
    ra2 = df_dim['rough ratio'].value_counts().to_dict()

    print(f'explict the square counts: {ra}')
    print(f'rough square counts: {ra2}')

    return df_dim

## use funcs

In [29]:
dimension_df('clean image/asian_art_sel')

explict the square counts: {'horizontal': 118, 'vertical': 71}
rough square counts: {'horizontal': 111, 'vertical': 62, 'square': 16}


,Object ID,width,height,ratio,rough ratio
0,126312,1500,2111,vertical,vertical
1,126722,1250,2757,vertical,vertical
2,127542,1400,2418,vertical,vertical
3,39317,2000,1604,horizontal,horizontal
4,39747,3811,3049,horizontal,horizontal
...,...,...,...,...,...
184,88303,1227,2439,vertical,vertical
185,88308,1250,2757,vertical,vertical
186,88350,1800,1636,horizontal,square
187,93922,1425,2450,vertical,vertical


In [30]:
dimension_df('clean image/full_selection_sel')

explict the square counts: {'vertical': 2186, 'horizontal': 378, 'square': 1}
rough square counts: {'vertical': 2081, 'horizontal': 295, 'square': 189}


,Object ID,width,height,ratio,rough ratio
0,100017,991,1795,vertical,vertical
1,100019,1800,1898,vertical,square
2,100020,1440,2526,vertical,vertical
3,100021,1663,1960,vertical,vertical
4,100024,1225,2838,vertical,vertical
...,...,...,...,...,...
2560,98751,2949,3892,vertical,vertical
2561,99061,1850,1885,vertical,square
2562,99070,1500,2250,vertical,vertical
2563,99509,3000,4000,vertical,vertical


overall most clothes images are vertical, but most asian clothes are horizontal. 